In [8]:
import geopy
import requests
import pandas as pd
from datetime import datetime
import logging
import pytz
import threading
import time
import sqlite3

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Conectar ao arquivo .db (ele será criado se não existir)
conn = sqlite3.connect('dados_ocorrencia_onibus.db')
cursor = conn.cursor()

# Criar tabela (se ainda não existir)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS onibus (
        ID INTEGER,
        TIPO INTEGER,
        LINHA INTEGER,
        MENSAGEM TEXT,
        HORA TEXT,
        FLAG_PROCES INTEGER
    )
''')
conn.commit()

# Função para buscar e processar os dados de uma linha de ônibus
def buscar_e_processar_dados():
    try:


        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")
        # URL da página com os dados
        url = "https://transporteservico.urbs.curitiba.pr.gov.br/getOcorrenciaCCOporLinha.php?c=821f0"  # Substitua com a URL correta
        
        # Fazer a requisição GET
        response = requests.get(url)
        
        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            dados = response.json()  # Converter o conteúdo para JSON
        
            # Criar um DataFrame com os dados
            df_codigos_onibus = pd.DataFrame(dados)
            df_codigos_onibus['HORA'] = hora_online
            return df_codigos_onibus
            
        if df_codigos_onibus.empty:
            logging.warning("Nenhum ônibus encontrado.")
            return None

        

    except requests.Timeout:
        logging.error("Timeout: A conexão demorou mais de 10 segundos.")
        return None
    except requests.RequestException as e:
        logging.error("Erro na requisição: %s", e)
        return None
    except Exception as e:
        logging.error("Erro ao processar os dados: %s", e)
        return None

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    for index, row in df_result.iterrows():
        cursor.execute('''
            INSERT INTO onibus (ID, TIPO, LINHA, MENSAGEM, HORA, FLAG_PROCES) 
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            row['ID'], row['TIPO'], row['LINHA'], row['MENSAGEM'],
            row['HORA'], 0
        ))
    conn.commit()

# Função principal para executar o processo
def main():
    
    df_concatenado = buscar_e_processar_dados()
    print(df_concatenado)
    if df_concatenado is not None:
        gravar_no_banco(df_concatenado)

 

# Iniciar a execução
if __name__ == "__main__":
    main()

conn.close()


        ID TIPO LINHA                 MENSAGEM                 HORA
0   573291   46   305          Possível atraso  2024-10-16 20:51:42
1   573325   20   550          Possível atraso  2024-10-16 20:51:42
2   573331   20   212          Possível atraso  2024-10-16 20:51:42
3   573334   20   821          Possível atraso  2024-10-16 20:51:42
4   573333   20   030          Possível atraso  2024-10-16 20:51:42
5   573332   30   902  Desvio na rota na linha  2024-10-16 20:51:42
6   573332   30   901  Desvio na rota na linha  2024-10-16 20:51:42
7   573332   30   307  Desvio na rota na linha  2024-10-16 20:51:42
8   573332   30   965  Desvio na rota na linha  2024-10-16 20:51:42
9   573332   30   967  Desvio na rota na linha  2024-10-16 20:51:42
10  573341   46   307          Possível atraso  2024-10-16 20:51:42
11  573337   30   231  Desvio na rota na linha  2024-10-16 20:51:42
12  573337   30   233  Desvio na rota na linha  2024-10-16 20:51:42
13  573337   30   237  Desvio na rota na linha  